In [ ]:
!pip install spacy_transformers
!pip install -U spacy

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

In [ ]:
cv_data=json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json', "r"))

In [ ]:
len(cv_data)

In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

In [ ]:
cv_data

In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()

    for text, annot in tqdm(data):
      doc = nlp.make_doc(text)
      annot = annot["entities"]

      ents = []
      entity_indices = []

      for start, end, label in annot:
        skip_entity = False
        for idx in range(start, end):
          if idx in entity_indices:
            skip_entity = True
            break
        if skip_entity == True:
          continue

        entity_indices = entity_indices + list(range(start,end))

        try:
          span = doc.char_span(start,end, label=label, alignment_mode="strict")
        except:
          continue

        if span is None:
          err_data = str([start, end]) + "     " + str(text) + "\n"
          file.write(err_data)
          # files throwing errors

        else:
          ents.append(span)

      try:
        doc.ents = ents
        db.add(doc)
      except:
        pass

    return db


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
file = open("error.txt", "w")

db = get_spacy_doc(file, train)
db.to_disk("train_data.spacy")

db = get_spacy_doc(file, test)
db.to_disk("test_data.spacy")


file.close()

In [ ]:
len(db.tokens)

In [ ]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy

In [ ]:
nlp = spacy.load("/content/output/model-best")

In [ ]:
!pip install PyMuPDF

In [ ]:
import sys, fitz

In [ ]:
fname = "/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf"
doc = fitz.open(fname)

In [ ]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [ ]:
text = text.strip()

In [ ]:
text = " ".join(text.split())

In [ ]:
text

In [ ]:
doc = nlp(text)

with open("output.txt", "w", encoding="utf-8") as file:
    for ent in doc.ents:
        file.write(f"{ent.text}   -->   {ent.label_}\n")